In [1]:
%matplotlib inline

# Aprender Pytorch con ejemplos - Control de flujo

Para mostrar el poder de los gráficos dinámicos de PyTorch, implementaremos un modelo muy extraño: un polinomio de tercer-quinto orden que en cada pasada hacia adelante elige un número aleatorio entre 3 y 5 y usa esa cantidad de órdenes, reutilizando los mismos pesos varias veces para calcular el cuarto y quinto orden.

In [2]:
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 1978.011474609375
3999 950.7560424804688
5999 459.6216735839844
7999 222.87266540527344
9999 109.45297241210938
11999 58.20433807373047
13999 32.66537857055664
15999 26.182247161865234
17999 14.369629859924316
19999 11.48314094543457
21999 10.150697708129883
23999 9.384397506713867
25999 9.145648956298828
27999 8.992356300354004
29999 8.906698226928711
Result: y = -0.008100783452391624 + 0.8540968298912048 x + 0.0009260551887564361 x^2 + -0.09319843351840973 x^3 + 8.663882181281224e-05 x^4 ? + 8.663882181281224e-05 x^5 ?
